# ITS_LIVE 
## Global Ice Velocities


---
> OpenApi endpoint https://staging.nsidc.org/apps/itslive-search/docs


In [1]:
#1: We are importing the its_live class and supresing non essential warnings
from itslive import itslive_ui
ui = itslive_ui('north')
import warnings
warnings.filterwarnings('ignore')

In [5]:
#2: Now let's render our UI and pick up an hemisphere, if you update the hemisphere you need to execute the cell again.
ui.render()

Dropdown(description='Hemisphere:', options=('global', 'south', 'north'), value='global')

SelectionRangeSlider(description='Date Range', index=(9849, 13149), layout=Layout(width='100%'), options=((' 1…

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [6]:
#3: We build the parameters to query the ITS_LIVE Search API, we get the time coverage for our selected area
params = ui.build_params()
print(f'current parameters: {params}')
timeline = None
if params is not None:
    timeline = ui.update_coverages()
    total =  sum(item['count'] for item in timeline)
    print(f'Total data granules: {total:,}')
timeline

current parameters: {'polygon': '-44.4141,59.8647,-44.4141,60.3465,-42.4805,60.3465,-42.4805,59.8647,-44.4141,59.8647', 'start': '2010-12-19', 'end': '2020-01-01', 'percent_valid_pixels': '10'}
Total data granules: 4,374


[{'count': 5, 'year': 2010},
 {'count': 102, 'year': 2011},
 {'count': 143, 'year': 2012},
 {'count': 465, 'year': 2013},
 {'count': 827, 'year': 2014},
 {'count': 601, 'year': 2015},
 {'count': 520, 'year': 2016},
 {'count': 857, 'year': 2017},
 {'count': 853, 'year': 2018},
 {'count': 1, 'year': 2019}]

In [7]:
#4: Now we are going to get the velocity pair urls
params = ui.build_params()
if params is not None:
    urls = ui.get_granule_urls()

In [9]:
#5: Print the first 10 granule URLs
urls[0:10]

[{'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32623/LC08_L1TP_001018_20130926_20170501_01_T1_X_LE07_L1TP_001018_20120814_20161129_01_T1_G0240V01_P045.nc'},
 {'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32623/LC08_L1TP_001018_20131215_20170427_01_T1_X_LE07_L1TP_001018_20120713_20161130_01_T1_G0240V01_P022.nc'},
 {'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32623/LC08_L1TP_001018_20131215_20170427_01_T1_X_LE07_L1TP_001018_20120814_20161129_01_T1_G0240V01_P029.nc'},
 {'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32623/LC08_L1TP_001018_20131215_20170427_01_T1_X_LE07_L1TP_001018_20131004_20161120_01_T1_G0240V01_P035.nc'},
 {'url': 'http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/velocity_image_pair/landsat/v00.0/32623/LC08_L1TP_001018_20131231_20170427_01_T1_X_LE07_L1TP_001018_20120814_20161129_0

In [ ]:
#6: This will query AWS(where the granules are stored) so we know the total size of our first N granules
# This may take some time, try reducing the selected area or constraining the other parameters to download a reasonable number of granules.
max_granules = 10000
sizes = ui.calculate_file_sizes(max_granules)
total_zise = round(sum(sizes)/1024,2)
print(f'Size to download for the first {max_granules:,} granules: {total_zise} MB')

## Downloading the data
**Now that we have our list of data granules we can download them from AWS.**

If this notebook is running inside AWS we could load the granules into a Dask cluster and reduce our processing times and costs.
Let's get some coffee, some data requests are in the Gigabytes realm and may take a little while to be processed. 
Once that your status URL says is completed we can grab the HDF5 data file using the URL on the same response!

In [ ]:
#7: Download the velocity granules